# Fileupload

In [6]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
import scipy
import plotly.express as px

In [7]:
 os.getcwd()

'/home/wseis/qmra/tools'

In [8]:
data = pd.read_csv("./logremoval/testdata.csv")

In [9]:
data

,date,inflow,censored_in,outflow,censored_out
0,01.01.2022,345678.0,NaN,3.0,NaN
1,02.01.2022,89905.0,NaN,4.0,NaN
2,03.01.2022,567878.0,NaN,5.0,NaN
3,04.01.2022,56789.0,NaN,6.0,NaN
4,05.01.2022,52535.0,NaN,7.0,NaN
5,06.01.2022,8765.0,NaN,8.0,NaN
6,07.01.2022,NaN,NaN,90.0,NaN
7,08.01.2022,2345678.0,NaN,54.0,NaN
8,09.01.2022,987654.0,NaN,NaN,NaN
9,10.01.2022,54322.0,NaN,23.0,NaN


In [10]:
data = data[["inflow", "outflow"]].dropna()

NameError: name 'n_paired' is not defined

In [13]:
data.count()["inflow"]

11

In [14]:
data.dropna()

,inflow,outflow
0,345678.0,3.0
1,89905.0,4.0
2,567878.0,5.0
3,56789.0,6.0
4,52535.0,7.0
5,8765.0,8.0
7,2345678.0,54.0
9,54322.0,23.0
10,9865.0,43.0
11,5432.0,543.0


In [15]:
Xin = np.ones_like(data["inflow"])
Xout = np.ones_like(data["inflow"])
nb_inflow = sm.NegativeBinomial(data["inflow"], Xin).fit()
nb_outflow = sm.NegativeBinomial(data["outflow"], Xout).fit()

Optimization terminated successfully.
         Current function value: 13.576425
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 8
Optimization terminated successfully.
         Current function value: 4.780766
         Iterations: 5
         Function evaluations: 8
         Gradient evaluations: 8


In [16]:
nb_inflow.params

const    12.927123
alpha     2.335708
dtype: float64

In [58]:
def simulate_negbin(model):
    mu = np.exp(model.params[0])
    p = 1/(1+np.exp(model.params[0])*model.params[1])
    n = np.exp(model.params[0])*p/(1-p)
    return np.random.negative_binomial(n, p, 10000)
    

In [59]:
inflow = simulate_negbin(nb_inflow)
outflow = simulate_negbin(nb_outflow)
df = pd.DataFrame({"inflow": inflow+1, "outflow": outflow+1})

In [60]:
df["outflow"] = df["outflow"]+1
df["inflow"] = df["inflow"]+1


In [61]:
df["login"] = np.log10(df["inflow"])

In [62]:
df["LRV"] =np.log10((df["inflow"])/df["outflow"])


In [63]:
df.describe()

,inflow,outflow,login,LRV
count,1.000000e+04,10000.000000,10000.000000,10000.000000
mean,4.280572e+05,64.697800,4.986912,3.633516
std,6.387981e+05,97.158351,1.062100,1.271978
min,2.000000e+00,2.000000,0.301030,-2.342423
25%,3.079275e+04,6.000000,4.488448,2.942730
50%,1.671295e+05,26.000000,5.223053,3.774518
75%,5.570605e+05,82.000000,5.745902,4.526836
max,5.769794e+06,1072.000000,6.761160,6.433687


In [65]:
q = df.quantile([.1, .5, .9])

In [73]:
q["LRV"]#.iloc[1]

0.1    1.974231
0.5    3.774518
0.9    5.127296
Name: LRV, dtype: float64